### Создание графа Support Network

In [1]:
import numpy as np
import re
from time import sleep
import chess 
import chess.pgn
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
class Cell: 
    def __init__(self, x, y, t, c): 
        self.x = x 
        self.y = y 
        self.type = t
        self.color = c
        # вектор с "координатами" клеток, попадющих под бой фигуры, стоящей на этой клетке
        self.attack = [] 
        # вектор фигур, атакующих эту клетку
        self.attacked = []
        # вектор с "координатами" клеток, попадющих под защиту фигуры, стоящей на этой клетке
        self.defend = []
        
def attack_by_black_pawn(pawn, cells): 
    if (pawn.x - 1 >= 0) and (pawn.y - 1 >= 0) and (cells[pawn.x - 1][pawn.y - 1].color != 0):
        if (pawn.color != cells[pawn.x - 1][pawn.y - 1].color):
            pawn.attack.append((cells[pawn.x - 1][pawn.y - 1].type, pawn.x - 1, pawn.y - 1))
        else:
            pawn.defend.append((cells[pawn.x - 1][pawn.y - 1].type, pawn.x - 1, pawn.y - 1))
        cells[pawn.x - 1][pawn.y - 1].attacked.append(pawn) 
        
    if (pawn.x - 1 >= 0) and (pawn.y + 1 < 8) and (cells[pawn.x - 1][pawn.y + 1].color != 0): 
        if (pawn.color != cells[pawn.x - 1][pawn.y + 1].color):
            pawn.attack.append((cells[pawn.x - 1][pawn.y + 1].type, pawn.x - 1, pawn.y + 1)) 
        else:
            pawn.defend.append((cells[pawn.x - 1][pawn.y + 1].type, pawn.x - 1, pawn.y + 1)) 
        cells[pawn.x - 1][pawn.y + 1].attacked.append(pawn)
        
def attack_by_white_pawn(pawn, cells): 
    if (pawn.x + 1 < 8) and (pawn.y - 1 >= 0) and (cells[pawn.x + 1][pawn.y - 1].color != 0): 
        if (pawn.color != cells[pawn.x + 1][pawn.y - 1].color):
            pawn.attack.append((cells[pawn.x + 1][pawn.y - 1].type, pawn.x + 1, pawn.y - 1)) 
        else:
            pawn.defend.append((cells[pawn.x + 1][pawn.y - 1].type, pawn.x + 1, pawn.y - 1))
        cells[pawn.x + 1][pawn.y - 1].attacked.append(pawn) 
        
    if (pawn.x + 1 < 8) and (pawn.y + 1 < 8) and (cells[pawn.x + 1][pawn.y + 1].color != 0): 
        if (pawn.color != cells[pawn.x + 1][pawn.y + 1].color):
            pawn.attack.append((cells[pawn.x + 1][pawn.y + 1].type, pawn.x + 1, pawn.y + 1)) 
        else:
            pawn.defend.append((cells[pawn.x + 1][pawn.y + 1].type, pawn.x + 1, pawn.y + 1))
        cells[pawn.x + 1][pawn.y + 1].attacked.append(pawn)

def attack_by_knight(knight, cells): 
    moves = [(-1, 2), 
             (-2, 1), 
             (-2, -1), 
             (-1, -2), 
             (1, -2), 
             (2, -1), 
             (2, 1), 
             (1, 2)]
    
    for move in moves: 
        if (knight.x + move[0] >= 0) and (knight.x + move[0] < 8) and (knight.y + move[1] >= 0) and (knight.y + move[1] < 8) and (cells[knight.x + move[0]][knight.y + move[1]].color != 0): 
            if (knight.color != cells[knight.x + move[0]][knight.y + move[1]].color):
                knight.attack.append((cells[knight.x + move[0]][knight.y + move[1]].type, knight.x + move[0], knight.y + move[1]))
            else:
                knight.defend.append((cells[knight.x + move[0]][knight.y + move[1]].type, knight.x + move[0], knight.y + move[1]))
            cells[knight.x + move[0]][knight.y + move[1]].attacked.append(knight)
            
def attack_by_bishop(bishop, cells): 
    move_x = bishop.x + 1 
    move_y = bishop.y + 1 
    
    while (move_x < 8) and (move_y < 8): 
        if (cells[move_x][move_y].color != 0):
            if (bishop.color != cells[move_x][move_y].color):
                bishop.attack.append((cells[move_x][move_y].type, move_x, move_y))
            else:
                bishop.defend.append((cells[move_x][move_y].type, move_x, move_y))
            cells[move_x][move_y].attacked.append(bishop) 
        
        if cells[move_x][move_y].type != 0: 
            break 
            
        move_x += 1 
        move_y += 1

    move_x = bishop.x + 1
    move_y = bishop.y - 1
    while (move_x < 8) and (move_y >= 0):
        if (cells[move_x][move_y].color != 0):
            if (bishop.color != cells[move_x][move_y].color):
                bishop.attack.append((cells[move_x][move_y].type, move_x, move_y))
            else:
                bishop.defend.append((cells[move_x][move_y].type, move_x, move_y))
            cells[move_x][move_y].attacked.append(bishop)
        
        if cells[move_x][move_y].type != 0:
            break
            
        move_x += 1
        move_y -= 1

    move_x = bishop.x - 1
    move_y = bishop.y - 1
    
    while (move_x >= 0) and (move_y >= 0):
        if (cells[move_x][move_y].color != 0):
            if (bishop.color != cells[move_x][move_y].color):
                bishop.attack.append((cells[move_x][move_y].type, move_x, move_y))
            else:
                bishop.defend.append((cells[move_x][move_y].type, move_x, move_y))
            cells[move_x][move_y].attacked.append(bishop)
        
        if cells[move_x][move_y].type != 0:
            break
            
        move_x -= 1
        move_y -= 1

    move_x = bishop.x - 1
    move_y = bishop.y + 1
    
    while (move_x >= 0) and (move_y < 8):
        if (cells[move_x][move_y].color != 0):
            if (bishop.color != cells[move_x][move_y].color):
                bishop.attack.append((cells[move_x][move_y].type, move_x, move_y))
            else:
                bishop.defend.append((cells[move_x][move_y].type, move_x, move_y))
            cells[move_x][move_y].attacked.append(bishop)
        
        if cells[move_x][move_y].type != 0:
            break
            
        move_x -= 1
        move_y += 1

        
def attack_by_rook(rook, cells): 
    move_x = rook.x + 1 
    
    while move_x < 8:
        if (cells[move_x][rook.y].color != 0):
            if (rook.color != cells[move_x][rook.y].color):
                rook.attack.append((cells[move_x][rook.y].type, move_x, rook.y)) 
            else:
                rook.defend.append((cells[move_x][rook.y].type, move_x, rook.y)) 
            cells[move_x][rook.y].attacked.append(rook) 
        
        if cells[move_x][rook.y].type != 0: 
            break 
        move_x += 1

    move_x = rook.x - 1
    
    while move_x >= 0:
        if (cells[move_x][rook.y].color != 0):
            if (rook.color != cells[move_x][rook.y].color):
                rook.attack.append((cells[move_x][rook.y].type, move_x, rook.y))
            else:
                rook.defend.append((cells[move_x][rook.y].type, move_x, rook.y))
            cells[move_x][rook.y].attacked.append(rook)
            
        if cells[move_x][rook.y].type != 0:
            break
        move_x -= 1

    move_y = rook.y + 1
    
    while move_y < 8:
        if (cells[rook.x][move_y].color != 0):
            if (rook.color != cells[rook.x][move_y].color):
                rook.attack.append((cells[rook.x][move_y].type, rook.x, move_y))
            else:
                rook.defend.append((cells[rook.x][move_y].type, rook.x, move_y))
            cells[rook.x][move_y].attacked.append(rook)
            
        if cells[rook.x][move_y].type != 0:
            break
        move_y += 1

    move_y = rook.y - 1
    
    while move_y >= 0:
        if (cells[rook.x][move_y].color != 0):
            if (rook.color != cells[rook.x][move_y].color):
                rook.attack.append((cells[rook.x][move_y].type, rook.x, move_y))
            else:
                rook.defend.append((cells[rook.x][move_y].type, rook.x, move_y))
            cells[rook.x][move_y].attacked.append(rook)
            
        if cells[rook.x][move_y].type != 0:
            break
        move_y -= 1

        
def attack_by_queen(queen, cells): 
    attack_by_rook(queen, cells) 
    attack_by_bishop(queen, cells)
    
    

def attack_by_king(king, cells): 
    moves = [(-1, 0), (-1, 1), (-1, -1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)] 
    
    for move in moves: 
        if (king.x + move[0] >= 0) and (king.x + move[0] < 8) and (king.y + move[1] >= 0) and (king.y + move[1] < 8) and (cells[king.x + move[0]][king.y + move[1]].color != 0):
            if (king.color != cells[king.x + move[0]][king.y + move[1]].color):
                king.attack.append((cells[king.x + move[0]][king.y + move[1]].type, king.x + move[0], king.y + move[1]))
            else:
                king.defend.append((cells[king.x + move[0]][king.y + move[1]].type, king.x + move[0], king.y + move[1]))
            cells[king.x + move[0]][king.y + move[1]].attacked.append(king)
            


In [3]:
def define_figures(board): 
    figures = [] 
    cells = [[] for _ in range(8)] 

    for i in range(8): 
        for j in range(8): 
            if board[i][j] == 0:
                cells[i].append(Cell(i, j, board[i][j], 0))
            elif board[i][j].islower():
                figures.append(Cell(i, j, board[i][j], -1))
                cells[i].append(Cell(i, j, board[i][j], -1))
            elif board[i][j].isupper():
                figures.append(Cell(i, j, board[i][j], 1))
                cells[i].append(Cell(i, j, board[i][j], 1))
                
    for figure in figures: 
        if figure.type == 'p': 
            attack_by_black_pawn(figure, cells)
        elif figure.type == 'P': 
            attack_by_white_pawn(figure, cells)
        elif figure.type == 'n' or figure.type == 'N': 
            attack_by_knight(figure, cells)
        elif figure.type == 'b' or figure.type == 'B': 
            attack_by_bishop(figure, cells) 
        elif figure.type == 'r' or figure.type == 'R': 
            attack_by_rook(figure, cells) 
        elif figure.type == 'q' or figure.type == 'Q': 
            attack_by_queen(figure, cells) 
        elif figure.type == 'k' or figure.type == 'K': 
            attack_by_king(figure, cells)
            
    return figures

In [4]:
def board_to_array(board):
    # Создаем пустой двумерный массив размером 8x8
    board_array = [[None for j in range(8)] for i in range(8)]

    # Проходим по каждой позиции на шахматной доске
    for row in range(8):
        for col in range(8):
            # Получаем фигуру с позиции и преобразуем её в символ
            piece = board.piece_at(8 * row + col)
            symbol = piece.symbol() if piece else 0

            # Сохраняем символ фигуры в двумерный массив
            board_array[row][col] = symbol

    return board_array

In [16]:
import chess
import chess.pgn
import networkx as nx
import pickle
import os
from tqdm.auto import tqdm

import subprocess

def create_graph(board, letters, graphs):
    board_array = board_to_array(board)
    figures = define_figures(board_array)
    G = nx.DiGraph()
    for figure in figures:
        for figure_attacked in figure.attack:
            first = str(figure.type) + letters[figure.y] + str(figure.x + 1)
            second = str(figure_attacked[0]) + letters[figure_attacked[2]] + str(figure_attacked[1] + 1)
            G.add_edge(first, second)

        for figure_defended in figure.defend:
            first = str(figure.type) + letters[figure.y] + str(figure.x + 1)
            second = str(figure_defended[0]) + letters[figure_defended[2]] + str(figure_defended[1] + 1)
            G.add_edge(first, second)

    graphs.append(G)

def count_games(input_file_path):
    # Команда, которую мы хотим выполнить в bash
    command = f'grep -o -w "Event" {input_file_path} | wc -l'

    # Выполнение команды и получение вывода
    output = subprocess.check_output(command, shell=True)

    # Преобразование вывода в строку и вывод результата
    count = int(output.decode('utf-8').strip())
    return count

def create_and_save_all_graphs(input_file_path, output_dir):
    if not os.path.exists(output_dir): 
        os.makedirs(output_dir)
    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    all_graphs = []  # Список для хранения всех графов
    total_games = 100

    with open(input_file_path) as pgn_file:
        for i in tqdm(range(total_games), desc="Processing games", unit="game"):
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            
            graphs = []
            board = game.board()
            create_graph(board, letters, graphs)
            for move_number, move in enumerate(game.mainline_moves()):
                board.push(move)
                create_graph(board, letters, graphs)
    
            temp = input_file_path.split('.')[0]
            output_path = f'{output_dir}/{temp}_{i}/{game.headers.get("White")}/{game.headers.get("Black")}/{game.headers.get("Result")}/{game.headers.get("WhiteElo")}/{game.headers.get("BlackElo")}/{game.headers.get("Opening")}/{game.headers.get("Variation")}.pickle'
            with open(output_path, "wb") as f:
                pickle.dump(graphs, f)


In [ ]:
with open("support_networks/database_5.pickle", 'rb') as f:
    graphs = pickle.load(f)

In [ ]:
colors = ["red" if node[0][0].isupper() else "black" for node in graphs[0].nodes(data=True)]
nx.draw(graphs[0], with_labels=True, node_color=colors)

In [ ]:
import os
import concurrent.futures

def save_output_to_file(output, file_name):
    with open(file_name, 'w') as f:
        f.write(output)


input_folder = os.getcwd() # Текущая директория
output_folder = 'support_network'

# Создаем папки для изображений и логов, если они ещё не существуют
os.makedirs(output_folder, exist_ok=True)

# Получаем список файлоw, начинающихся с "database_part"
files = [f.split('/')[-1] for f in os.listdir(input_folder) if f.startswith('database_part')]

# Используем ThreadPoolExecutor для выполнения функций в параллельных потоках
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Отправляем задачи на выполнение
    future_to_file = {executor.submit(create_and_save_all_graphs, file, output_folder): file for file in files}

    # Обрабатываем результаты выполнения
    for future in concurrent.futures.as_completed(future_to_file):
        file = future_to_file[future]
        try:
            output = future.result()
        except Exception as exc:
            print(f'Файл {file} вызвал исключение: {exc}')
        else:
            print("успех")